In [1]:
#!pip install --user tensorflow
#!pip install --user sklearn
#!pip install --user lime
#!pip install --user tf-explain
#!pip install --user opencv-python

In [2]:
import tensorflow as tf
#import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle

2023-05-25 15:53:02.550212: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-25 15:53:03.966546: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.3/lib64:/scratch/ponel/python/lib:/usr/local/cuda-11.3/tensorRT-8.0/lib64:/usr/local/cuda-11.3/cudnn-8.2/lib64:/usr/local/cuda-11.3/nccl-2.9/lib64
2023-05-25 15:53:03.966697: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: 

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

## Random imports go!

In [4]:
!nvidia-smi

Thu May 25 15:53:08 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.60.13    Driver Version: 525.60.13    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:B1:00.0 Off |                  N/A |
| 22%   36C    P2    72W / 250W |  10111MiB / 11264MiB |     28%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:B2:00.0 Off |                  N/A |
| 22%   

In [5]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Flatten

In [6]:
from functools import partial
from sklearn.model_selection import train_test_split

from sklearn import tree
from graphviz import Source

In [7]:
from tf_explain.core import GradCAM, IntegratedGradients, VanillaGradients, GradientsInputs, OcclusionSensitivity
from tf_explain.callbacks.grad_cam import GradCAMCallback
from tf_explain.callbacks.vanilla_gradients import VanillaGradientsCallback
from tf_explain.callbacks.gradients_inputs import GradientsInputsCallback
from tf_explain.callbacks.occlusion_sensitivity import OcclusionSensitivityCallback

## Load the Dataset

In [8]:
img_dim_x = 28
img_dim_y = 28
img_dim_z = 1

batch_size = 1024

In [9]:
def load_MNIST(dirpath='/data/project/FoolingDetection/mnist.npz'):
    with np.load(dirpath) as data:
        x_train, y_train = data['x_train'], data['y_train']
        x_test, y_test = data['x_test'], data['y_test']

    x_train = x_train/255
    x_test = x_test/255

    x_train = x_train.reshape(-1, 28, 28, 1)
    x_test = x_test.reshape(-1, 28, 28, 1)

    return x_train, y_train, x_test, y_test

In [10]:
def load_CIFAR_batch(filename):
    """Load a single batch of the CIFAR-10 dataset"""
    with open(filename, 'rb') as f:
        datadict = pickle.load(f, encoding='bytes')
        X = datadict[b'data']
        Y = datadict[b'labels']
        X = X.reshape(10000, 3, 32, 32).transpose(0, 2, 3, 1).astype("float")
        Y = np.array(Y)
        return X, Y

def load_CIFAR10(dirpath="/data/project/FoolingDetection/cifar-10-batches-py"):
    """Load all batches of the CIFAR-10 dataset from a directory"""
    xs = []
    ys = []
    for b in range(1, 5):
        f = os.path.join(dirpath, 'data_batch_%d' % (b,))
        X, Y = load_CIFAR_batch(f)
        xs.append(X)
        ys.append(Y)
    Xtr = np.concatenate(xs)
    Ytr = np.concatenate(ys)
    Xte, Yte = load_CIFAR_batch(os.path.join(dirpath, 'test_batch'))
    return Xtr, Ytr, Xte, Yte

In [11]:
def to_rgb(x):
    x_rgb = np.zeros((x.shape[0], img_dim_x, img_dim_y, img_dim_z))
    for i in range(3):
        x_rgb[..., i] = x[..., 0]
    return x_rgb

#x_train_rgb = to_rgb(x_train)
#x_test_rgb = to_rgb(x_test)

In [12]:
x_train, y_train, x_test, y_test = load_MNIST()
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))

## Stop early!

In [13]:
early_stopping = tf.keras.callbacks.EarlyStopping(
  monitor="val_loss",
  patience=3,  
  restore_best_weights=True  
)

## Some Hyperparameters

In [14]:
### Hyperparameters
# Instantiate an optimizer, initializer, regularizer, loss_fn
optimizer = tf.keras.optimizers.Adam()
initializer = tf.keras.initializers.HeNormal()
regularizer = tf.keras.regularizers.L2(1e-4)
loss_fn = tf.losses.SparseCategoricalCrossentropy()

## I C an NN Model

In [15]:
callbacks=[
    GradCAMCallback(class_index=0, validation_data=(x_test, y_test)),
    VanillaGradientsCallback(class_index=0, validation_data=(x_test, y_test)),
    GradientsInputsCallback(class_index=0, validation_data=(x_test, y_test)),
    # OcclusionSensitivityCallback(class_index=0, patch_size=4, validation_data=(x_test, y_test))
]

In [16]:
# Define a distribution strategy that uses only GPUs 0 and 1
strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])

def get_model_raw():
    with strategy.scope():
        ### Hyperparameters
        # Instantiate an optimizer, initializer, regularizer, loss_fn
        optimizer = tf.keras.optimizers.Adam()
        initializer = tf.keras.initializers.HeNormal()
        regularizer = tf.keras.regularizers.L2(1e-4)
        loss_fn = tf.losses.SparseCategoricalCrossentropy()
        metrics=tf.keras.metrics.SparseCategoricalAccuracy()

        # Alternative model from kaggle:
        model = keras.Sequential()

        model.add(layers.Conv2D(img_dim_x, (3,3), padding='same', activation='relu', input_shape=(img_dim_x,img_dim_y,img_dim_z)))
        model.add(layers.BatchNormalization())
        model.add(layers.Conv2D(img_dim_x, (3,3), padding='same', activation='relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.MaxPooling2D(pool_size=(2,2)))
        model.add(layers.Dropout(0.3))

        model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.MaxPooling2D(pool_size=(2,2)))
        model.add(layers.Dropout(0.5))

        model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.MaxPooling2D(pool_size=(2,2)))
        model.add(layers.Dropout(0.5))

        model.add(layers.Flatten())
        model.add(layers.Dense(128, activation='relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.5))
        model.add(layers.Dense(10))
        model.add(layers.Activation('softmax'))
        
        model.compile(loss=loss_fn, optimizer='adam', metrics=[metrics])
        return model
model = get_model_raw()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:1',)
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/

In [31]:
# model.fit(x_train, y_train, epochs=2, batch_size=128, validation_data=(x_test,y_test), callbacks=callbacks)
model.fit(x_train, y_train, epochs=100, batch_size=128, validation_data=(x_test,y_test), callbacks=[early_stopping])

2023-05-25 15:49:20.930093: W tensorflow/tsl/framework/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 179.44MiB (rounded to 188160000)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-05-25 15:49:20.930593: W tensorflow/tsl/framework/bfc_allocator.cc:492] ****________________________________________________________________________________________________


InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

# Comparison 

## Naive approach: Take two images (or heatmap) (with same dimensions) and calculate different similarity metrics for them

In [19]:
from scipy.spatial import distance
from skimage.metrics import structural_similarity as ssim
import matplotlib.image as mpimg
from sklearn.metrics.pairwise import cosine_similarity

In [20]:
def euclidean_distance(im1, im2):
    im1 = im1.flatten()
    im2 = im2.flatten()
    return np.linalg.norm(im1 - im2)

def manhattan_distance(im1, im2):
    im1 = im1.flatten()
    im2 = im2.flatten()
    return distance.cityblock(im1, im2)
  
def minkowski_distance(im1, im2, p): # p=1 <-> Manhatten, p=2 Euclidian
    im1 = im1.flatten()
    im2 = im2.flatten()
    return distance.minkowski(im1, im2, p)

def chebyshev_distance(im1, im2):
    im1 = im1.flatten()
    im2 = im2.flatten()
    return distance.chebyshev(im1, im2)

def cosine_sim(im1, im2):
    im1 = im1.flatten()
    im2 = im2.flatten()
    return cosine_similarity(im1.reshape(1, -1), im2.reshape(1, -1))

def compute_ssim(im1, im2, multichannel=True):
    return ssim(im1, im2, multichannel=multichannel)

def run_all_distance_metrics(p1, p2):
  minkowski_p = 3
  return {"euclidean:": euclidean_distance(p1,p2),
          "manhatten:": manhattan_distance(p1,p2),
          "minkowski-{}".format(minkowski_p): minkowski_distance(p1, p2, minkowski_p),
          "chebyshev": chebyshev_distance(p1,p2),
          "cosine sim" : cosine_sim(p1,p2)
          }

# Running various techniques on more data

## Run the different explainers

In [21]:
# Function to create a dict with key=class values=data
def create_dict_classes_datapoints(x_test, y_test):
    class_dict = {}
    for i in range(len(x_test)):
        key = y_test[i]
        if key in class_dict:
            class_dict[key].append(x_test[i])
        else:
            class_dict[key] = [x_test[i]]
    return class_dict

# Define sorted labels 
unique_sorted_labels = sorted(set(map(str, y_test)), key=lambda x: (not x.isdigit(), int(x) if x.isdigit() else x))
images_by_class = create_dict_classes_datapoints(x_test, y_test)

In [22]:
def run_explainers(x_in, y_in, model, NUM_IMAGES=300):

    explainer_grad = GradCAM()
    grids_gradcam = []
    grids_gradcam_guided = []
    explainer_ig = IntegratedGradients()
    grids_ig = []
    explainer_g_t_i = GradientsInputs()
    grids_g_t_i = []
    explainer_vgrads = VanillaGradients()
    grids_vgrads = []
    explainer_occlusion = OcclusionSensitivity()
    grids_occlusion = []

    # Iterate over the test_dataset for NUM_IMAGES. Run all the explanations using tf-explain and store the results in lists to be accessed later
    for index, (x_example, y_example) in enumerate(zip(x_in, y_in)):
      if index == NUM_IMAGES:
        break
      print("image: {}".format(index))
      x_example = ([x_example], None)
      y_example_index = int(y_example)

      grids_gradcam.append(explainer_grad.explain(x_example, model, class_index=y_example_index, use_guided_grads=False))
      grids_gradcam_guided.append(explainer_grad.explain(x_example, model, class_index=y_example_index, use_guided_grads=True))
      grids_ig.append(explainer_ig.explain(x_example, model, class_index=y_example_index))
      grids_g_t_i.append(explainer_g_t_i.explain(x_example, model, class_index=y_example_index))
      grids_vgrads.append(explainer_vgrads.explain(x_example, model, class_index=y_example_index))
      grids_occlusion.append(explainer_occlusion.explain(x_example, model, class_index=y_example_index, patch_size=7))
    
    return [(grids_gradcam, "gradcam"), 
              (grids_gradcam_guided, "gradcam_guided"), 
              (grids_ig, "ig"), 
              (grids_g_t_i, "g_t_i"), 
              (grids_vgrads, "vgrads"), 
              (grids_occlusion, "occlusion")]

In [23]:
### Now store the grids on the local disk so we do not have to re-run this everytime


# grid_list needs to match definiton in run_explainations()
def save_grids_to_disk(grids_list, save_dir, dataset, model, attack="correct", override=False):

    # Define save directory
    save_dir = os.path.join(save_dir, dataset, model, attack)
    
    if os.path.exists(save_dir) and not override:
        raise Exception("The directory already exists and `override=False` was passed. Please make sure that you want to do this, this will delete any previous results.")
    
    # Iterate over all grids and save them to corresponding subfolder
    for grids, subfolder in grids_list:
        subfolder_path = os.path.join(save_dir, subfolder)

        # Create subdirectory if it does not exist
        if not os.path.exists(subfolder_path):
            os.makedirs(subfolder_path)

        # Save each grid as an image
        for i, grid in enumerate(grids):
            plt.imshow(grid, cmap='hot', interpolation='nearest')
            plt.savefig(os.path.join(subfolder_path, f"{i}.png"))
            plt.close()

In [24]:
# New function to run the metrics which supports batches of images
def run_all_distance_metrics_batch(P1, P2):
    minkowski_p = 3
    n = len(P1)
    result = {"euclidean": 0,
              "manhattan": 0,
              "minkowski-{}".format(minkowski_p): 0,
              "chebyshev": 0,
              "cosine sim": 0}
    for p1, p2 in zip(P1, P2):
        result["euclidean"] += euclidean_distance(p1,p2)
        result["manhattan"] += manhattan_distance(p1,p2)
        result["minkowski-{}".format(minkowski_p)] += minkowski_distance(p1, p2, minkowski_p)
        result["chebyshev"] += chebyshev_distance(p1,p2)
        result["cosine sim"] += cosine_sim(p1,p2)
    for key in result:
        result[key] /= n
    return result

In [ ]:
plt.imshow(x_test[0])

In [ ]:
# Printing an Occlusion sample to show which channels containw which information:
# Longer Comment inc:
# Sooo I looked into the documentation for this tf-explain module (https://github.com/sicara/tf-explain/blob/9d7d1e900ec3e3e4b5338fbc43dfb93539acecc2/tf_explain/core/occlusion_sensitivity.py#L64)
# Turns out, for an to me unknown reason they resize their output from a n-d image to (always) an 3-d image. 
# This is why I'm just going to average the values of the 3 maps, because essentially, this is what they orginate from

avg_grids_gradcam = [np.mean(image, axis=2) for image in grids_gradcam]
# avg_grids_occlusion = [np.mean(image, axis=2) for image in grids_occlusion] # Average the channels of Occlusion to one channel
# avg_grids_gradcam = [image[:,:,1] for image in grids_gradcam]
avg_grids_occlusion = [image[:,:,2] for image in grids_occlusion]



plt.imshow(grids_occlusion[0][:,:,2], cmap='Blues')
print(grids_occlusion[0][:, 14, 0])

## Same images

### IG vs. Vanilla Grads

In [ ]:
metrics_ig_vs_vgrads = run_all_distance_metrics_batch(grids_ig, grids_vgrads)
print(metrics_ig_vs_vgrads)

### IG vs. Gradients*inputs

In [ ]:
metrics_ig_vs_gti = run_all_distance_metrics_batch(grids_ig, grids_g_t_i)
print(metrics_ig_vs_gti)

### IG vs. GradCAM

In [ ]:
metrics_ig_vs_gradcam = run_all_distance_metrics_batch(grids_ig, avg_grids_gradcam)
print(metrics_ig_vs_gradcam)

### IG vs. Occlusion

In [ ]:
metrics_ig_vs_occlusion = run_all_distance_metrics_batch(grids_ig, avg_grids_occlusion)
print(metrics_ig_vs_occlusion)

### Vanilla Grads vs. Gradients*Inputs



In [ ]:
metrics_vgrads_vs_gti = run_all_distance_metrics_batch(grids_vgrads, grids_g_t_i)
print(metrics_vgrads_vs_gti)

### Vanilla Grads vs. GradCAM

In [ ]:
metrics_vgrads_vs_gradcam = run_all_distance_metrics_batch(grids_vgrads, avg_grids_gradcam)
print(metrics_vgrads_vs_gradcam)

### Vanilla Grads vs. Occlusion

In [ ]:
metrics_vgrads_vs_occlusion = run_all_distance_metrics_batch(grids_vgrads, avg_grids_occlusion)
print(metrics_vgrads_vs_occlusion)

### Gradients*Inputs vs. GradCAM

In [ ]:
metrics_gi_vs_gradcam = run_all_distance_metrics_batch(grids_g_t_i, avg_grids_gradcam)
print(metrics_gi_vs_gradcam)

### Gradients*Inputs vs. Occlusion

In [ ]:
metrics_gi_vs_occlusion = run_all_distance_metrics_batch(grids_g_t_i, avg_grids_occlusion)
print(metrics_gi_vs_occlusion)

### GradCAM vs. Occlusion

In [ ]:
metrics_gradcam_vs_occlusion = run_all_distance_metrics_batch(avg_grids_gradcam, avg_grids_occlusion)
print(metrics_gradcam_vs_occlusion)

## Different images 
Here we take the batches of images and set one batch of by a constant _shift_ s. This means that now no longer the same indexes of two gridlists correspond to the same image

In [ ]:
def rearrange_images(list_of_images, s):
    n = len(grids_ig)
    new_grids_ig = [None] * n
    for i in range(n):
        new_grids_ig[(i + s) % n] = grids_ig[i]
    return new_grids_ig

grids_ig_offset = rearrange_images(grids_ig, 1)
grids_vgrads_offset = rearrange_images(grids_vgrads, 1)
grids_g_t_i_offset = rearrange_images(grids_g_t_i, 1)
avg_grids_gradcam_offset = rearrange_images(avg_grids_gradcam, 1) # Average the channels of GradCAM to one channel


### IG vs. Vanilla Grads

In [ ]:
metrics_ig_vs_vgrads_diff = run_all_distance_metrics_batch(grids_ig_offset, grids_vgrads)
print(metrics_ig_vs_vgrads_diff)

### IG vs. Gradients*Inputs

In [ ]:
metrics_ig_vs_gti_diff = run_all_distance_metrics_batch(grids_ig_offset, grids_g_t_i)
print(metrics_ig_vs_gti_diff)

### IG vs. GradCAM

In [ ]:
metrics_ig_vs_gradcam_diff = run_all_distance_metrics_batch(grids_ig_offset, avg_grids_gradcam)
print(metrics_ig_vs_gradcam_diff)

### IG vs. Occlusion

In [ ]:
metrics_ig_vs_occlusion_diff = run_all_distance_metrics_batch(grids_ig_offset, avg_grids_occlusion)
print(metrics_ig_vs_occlusion_diff)

### Vanilla Grads vs. Gradients*Inputs



In [ ]:
metrics_vgrads_vs_gti_diff = run_all_distance_metrics_batch(grids_vgrads_offset, grids_g_t_i)
print(metrics_vgrads_vs_gti_diff)

### Vanilla Grads vs. GradCAM

In [ ]:
metrics_vgrads_vs_gradcam_diff = run_all_distance_metrics_batch(grids_vgrads_offset, avg_grids_gradcam)
print(metrics_vgrads_vs_gradcam_diff)

### Vanilla Grads vs. Occlusion

In [ ]:
metrics_vgrads_vs_occlusion_diff = run_all_distance_metrics_batch(grids_vgrads_offset, avg_grids_occlusion)
print(metrics_vgrads_vs_occlusion_diff)

### Gradients*Inputs vs. GradCAM

In [ ]:
metrics_gi_vs_gradcam_diff = run_all_distance_metrics_batch(grids_g_t_i_offset, avg_grids_gradcam)
print(metrics_gi_vs_gradcam_diff)

### Gradients*Inputs vs. Occlusion

In [ ]:
metrics_gi_vs_occlusion_diff = run_all_distance_metrics_batch(grids_g_t_i_offset, avg_grids_occlusion)
print(metrics_gi_vs_occlusion_diff)

### GradCAM vs. Occlusion

In [ ]:
metrics_gradcam_vs_occlusion_diff = run_all_distance_metrics_batch(avg_grids_gradcam_offset, avg_grids_occlusion)
print(metrics_gradcam_vs_occlusion_diff)

## Comparison: Same and Different images



In [ ]:
print("IG vs. Vanilla Gradients:")
print("same: {}".format(metrics_ig_vs_vgrads))
print("diff: {}".format(metrics_ig_vs_vgrads_diff))
print("------------------------------------------")
print("IG vs. G*I:")
print("same: {}".format(metrics_ig_vs_gti))
print("diff: {}".format(metrics_ig_vs_gti_diff))
print("------------------------------------------")
print("IG vs. GradCAM:")
print("same: {}".format(metrics_ig_vs_gradcam))
print("diff: {}".format(metrics_ig_vs_gradcam_diff))
print("------------------------------------------")
print("IG vs. Occlusion:")
print("same: {}".format(metrics_ig_vs_occlusion))
print("diff: {}".format(metrics_ig_vs_occlusion_diff))
print("------------------------------------------")
print("vgrads vs. G*I:")
print("same: {}".format(metrics_vgrads_vs_gti))
print("diff: {}".format(metrics_vgrads_vs_gti_diff))
print("------------------------------------------")
print("vgrads vs. GradCAM:")
print("same: {}".format(metrics_vgrads_vs_gradcam))
print("diff: {}".format(metrics_vgrads_vs_gradcam_diff))
print("------------------------------------------")
print("vgrads vs. Occlusion:")
print("same: {}".format(metrics_vgrads_vs_occlusion))
print("diff: {}".format(metrics_vgrads_vs_occlusion_diff))
print("------------------------------------------")
print("G*I vs. GradCAM:")
print("same: {}".format(metrics_gi_vs_gradcam))
print("diff: {}".format(metrics_gi_vs_gradcam_diff))
print("------------------------------------------")
print("G*I vs. Occlusion:")
print("same: {}".format(metrics_gi_vs_occlusion))
print("diff: {}".format(metrics_gi_vs_occlusion_diff))
print("------------------------------------------")
print("GradCAM vs. Occlusion:")
print("same: {}".format(metrics_gradcam_vs_occlusion))
print("diff: {}".format(metrics_gradcam_vs_occlusion_diff))
print("------------------------------------------")



# Random label test

In [26]:
import random
def generate_random_labels(y):
    unique_values = list(set(y))
    return [random.choice(unique_values) for _ in y]

y_train_random = np.asarray(generate_random_labels(y_train))

In [ ]:
model_random_labels = get_model_raw()
model_random_labels.fit(x_train, y_train_random, epochs=100, batch_size=128, validation_data=(x_test,y_test),callbacks=[early_stopping])

Epoch 1/100


2023-05-25 16:57:33.178577: W tensorflow/core/framework/dataset.cc:769] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.
2023-05-25 16:57:38.600720: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_2/dropout_8/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


 42/469 [=>............................] - ETA: 8s - loss: 3.4519 - sparse_categorical_accuracy: 0.0913

In [ ]:
explanations = run_explainers(x_test, y_test, model_random_labels, 300)

In [ ]:
save_grids_to_disk(grids_list=explanations, save_dir="/project/FoolingDetection/grids", dataset="mnist", model="cnn", attack="random_labels", override=True)

# Model Parameter Randomization Test (Indepdent)

In [27]:
# Needs to be run for longer.

def run_random_model_param_test_indep():
    # 1. Do training 
    model_random_params = get_model_raw()
    model_random_params.fit(x_train, y_train, epochs=3, batch_size=128, validation_data=(x_test,y_test),callbacks=[early_stopping])
    
    # Loop over layers: Randomize one layer at a time
    original_weights = [layer.get_weights() for layer in model_random_params.layers]
    for i in range(len(model_random_params.layers)-1, -1, -1):
        # Reset the weights to their original values
        for j, layer in enumerate(model_random_params.layers):
            layer.set_weights(original_weights[j])

        # Randomize the weights of the current layer if it has weights
        layer = model_random_params.layers[i]
        weights = layer.get_weights()
        if weights:  # Check if the layer has weights
            randomized_weights = [np.random.rand(*w.shape) for w in weights]
            layer.set_weights(randomized_weights)
            layer_name = layer.name
            grids = run_explainers(x_test, y_test, model_random_params, 300)
            save_grids_to_disk(grids_list=grids, save_dir="/project/FoolingDetection/grids", dataset="mnist", model="cnn", attack="RandParams/Independent/{}_{}".format(i, layer_name), override=False)
        # Now you can generate your saliency maps and observe the impact of the weight randomization
        # ...
    


In [28]:
run_random_model_param_test_indep()

Epoch 1/3


2023-05-25 15:57:36.458793: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_3/dropout_12/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


469/469 [==============================] - 20s 26ms/step - loss: 0.3900 - sparse_categorical_accuracy: 0.8796 - val_loss: 2.6590 - val_sparse_categorical_accuracy: 0.3857
Epoch 2/3
468/469 [============================>.] - ETA: 0s - loss: 0.0942 - sparse_categorical_accuracy: 0.9712

2023-05-25 15:58:01.570811: W tensorflow/core/framework/dataset.cc:769] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


469/469 [==============================] - 11s 23ms/step - loss: 0.0942 - sparse_categorical_accuracy: 0.9712 - val_loss: 0.0285 - val_sparse_categorical_accuracy: 0.9901
Epoch 3/3
469/469 [==============================] - 11s 23ms/step - loss: 0.0667 - sparse_categorical_accuracy: 0.9801 - val_loss: 0.0268 - val_sparse_categorical_accuracy: 0.9916
image: 0
1/1 [==============================] - 1s 1s/step
image: 1
1/1 [==============================] - 0s 93ms/step
image: 2
1/1 [==============================] - 0s 100ms/step
image: 3
1/1 [==============================] - 0s 94ms/step
image: 4
1/1 [==============================] - 0s 83ms/step
image: 5
1/1 [==============================] - 0s 100ms/step
image: 6
1/1 [==============================] - 0s 93ms/step
image: 7
1/1 [==============================] - 0s 80ms/step
image: 8
1/1 [==============================] - 0s 89ms/step
image: 9
1/1 [==============================] - 0s 96ms/step
image: 10
1/1 [=======================

...

image: 1
1/1 [==============================] - 0s 94ms/step
image: 2
1/1 [==============================] - 0s 95ms/step
image: 3
1/1 [==============================] - 0s 102ms/step
image: 4
1/1 [==============================] - 0s 97ms/step
image: 5
1/1 [==============================] - 0s 94ms/step
image: 6
1/1 [==============================] - 0s 155ms/step
image: 7
1/1 [==============================] - 0s 92ms/step
image: 8
1/1 [==============================] - 0s 95ms/step
image: 9
1/1 [==============================] - 0s 92ms/step
image: 10
1/1 [==============================] - 0s 81ms/step
image: 11
1/1 [==============================] - 0s 94ms/step
image: 12
1/1 [==============================] - 0s 96ms/step
image: 13
1/1 [==============================] - 0s 92ms/step
image: 14
1/1 [==============================] - 0s 89ms/step
image: 15
1/1 [==============================] - 0s 95ms/step
image: 16
1/1 [==============================] - 0s 84ms/step
image: 17
1/1 [

2023-05-25 16:05:49.634859: W tensorflow/core/framework/dataset.cc:769] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


image: 1
1/1 [==============================] - 0s 93ms/step
image: 2
1/1 [==============================] - 0s 94ms/step
image: 3
1/1 [==============================] - 0s 93ms/step
image: 4
1/1 [==============================] - 0s 95ms/step
image: 5
1/1 [==============================] - 0s 92ms/step
image: 6
1/1 [==============================] - 0s 91ms/step
image: 7
1/1 [==============================] - 0s 94ms/step
image: 8
1/1 [==============================] - 0s 96ms/step
image: 9
1/1 [==============================] - 0s 98ms/step
image: 10
1/1 [==============================] - 0s 82ms/step
image: 11
1/1 [==============================] - 0s 91ms/step
image: 12
1/1 [==============================] - 0s 94ms/step
image: 13
1/1 [==============================] - 0s 92ms/step
image: 14
1/1 [==============================] - 0s 94ms/step
image: 15
1/1 [==============================] - 0s 93ms/step
image: 16
1/1 [==============================] - 0s 96ms/step
image: 17
1/1 [==

2023-05-25 16:06:22.754835: W tensorflow/core/framework/dataset.cc:769] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


image: 1
1/1 [==============================] - 0s 92ms/step
image: 2
1/1 [==============================] - 0s 96ms/step
image: 3
1/1 [==============================] - 0s 81ms/step
image: 4
1/1 [==============================] - 0s 130ms/step
image: 5
1/1 [==============================] - 0s 91ms/step
image: 6
1/1 [==============================] - 0s 96ms/step
image: 7
1/1 [==============================] - 0s 96ms/step
image: 8
1/1 [==============================] - 0s 93ms/step
image: 9
1/1 [==============================] - 0s 90ms/step
image: 10
1/1 [==============================] - 0s 97ms/step
image: 11
1/1 [==============================] - 0s 96ms/step
image: 12
1/1 [==============================] - 0s 93ms/step
image: 13
1/1 [==============================] - 0s 95ms/step
image: 14
1/1 [==============================] - 0s 94ms/step
image: 15
1/1 [==============================] - 0s 94ms/step
image: 16
1/1 [==============================] - 0s 97ms/step
image: 17
1/1 [=

# Model Parameter Randomization Test (Cascading)

In [ ]:
# Needs to be run for longer.

def run_random_model_param_test_cascading():
    # 1. Do training 
    model_random_params = get_model_raw()
    model_random_params.fit(x_train, y_train, epochs=3, batch_size=128, validation_data=(x_test,y_test),callbacks=[early_stopping])
    
    # Loop over layers: Randomize one layer at a time
    original_weights = [layer.get_weights() for layer in model_random_params.layers]
    for i in range(len(model_random_params.layers)-1, -1, -1):
        # Randomize the weights of the current layer if it has weights
        layer = model_random_params.layers[i]
        weights = layer.get_weights()
        if weights:  # Check if the layer has weights
            randomized_weights = [np.random.rand(*w.shape) for w in weights]
            layer.set_weights(randomized_weights)
            layer_name = layer.name
            grids = run_explainers(x_test, y_test, model_random_params, 300)
            save_grids_to_disk(grids_list=grids, save_dir="/project/FoolingDetection/grids", dataset="mnist", model="cnn", attack="RandParams/Cascading/{}_{}".format(i, layer_name), override=False)
        # Now you can generate your saliency maps and observe the impact of the weight randomization
        # ...
    


In [ ]:
run_random_model_param_test_cascading()

# extra

In [ ]:
### Indepent Layer Randomization:

# Save the original weights
original_weights = [layer.get_weights() for layer in model.layers]

for i in range(len(model.layers)-1, -1, -1):
    # Reset the weights to their original values
    for j, layer in enumerate(model.layers):
        layer.set_weights(original_weights[j])
    
    # Randomize the weights of the current layer if it has weights
    layer = model.layers[i]
    weights = layer.get_weights()
    if weights:  # Check if the layer has weights
        randomized_weights = [np.random.rand(*w.shape) for w in weights]
        layer.set_weights(randomized_weights)
    
    # Now you can generate your saliency maps and observe the impact of the weight randomization
    # ...
    

In [ ]:
### Cascading Layer Randomization
original_weights = [layer.get_weights() for layer in model.layers]

for i in range(len(model.layers)-1, -1, -1):
    # Randomize the weights of the current layer if it has weights
    layer = model.layers[i]
    weights = layer.get_weights()
    if weights:  # Check if the layer has weights
        randomized_weights = [np.random.rand(*w.shape) for w in weights]
        layer.set_weights(randomized_weights)
        # Update the original_weights list with the new randomized weights
        original_weights[i] = randomized_weights
    
    # Now you can generate your saliency maps and observe the impact of the weight randomization
    # ...

In [ ]:
def generate_random_labels(y_test):
    unique_values = list(set(y_test))
    return [random.choice(unique_values) for _ in y_test]

# Comparing the results 

In [ ]:
from PIL import Image

# Dictionaries to hold our images
correct = {}
layer_random_indep = {}
layer_random_cascading = {}
labels_random = {}

# Directories
dir_correct = os.path.join(save_dir, 'correct')
dir_layer_random_indep = os.path.join(save_dir, 'layer_random_indep')
dir_layer_random_cascading = os.path.join(save_dir, 'layer_random_cascading')
dir_labels_random = os.path.join(save_dir, 'labels_random')

# Function to load images from a directory into a dictionary
def load_images(dir_path, img_dict):
    for sub_dir in os.listdir(dir_path):
        sub_dir_path = os.path.join(dir_path, sub_dir)
        if os.path.isdir(sub_dir_path):
            img_dict[sub_dir] = []
            for file in os.listdir(sub_dir_path):
                if file.endswith('.png'): # change this to the format of your images
                    file_path = os.path.join(sub_dir_path, file)
                    image = Image.open(file_path)
                    img_dict[sub_dir].append(image)

# Loading images into dictionaries
load_images(dir_correct, correct)
load_images(dir_layer_random_indep, layer_random_indep)
load_images(dir_layer_random_cascading, layer_random_cascading)
load_images(dir_labels_random, labels_random)


In [ ]:
# Verify that all dictionaries share the same keys
assert set(correct.keys()) == set(layer_random_indep.keys()) == set(layer_random_cascading.keys()) == set(labels_random.keys())

results = {
    "correct_layer_random_indep": {},
    "correct_layer_random_cascading": {},
    "correct_labels_random": {}
}

for key in correct.keys():
    result_indep = run_all_distance_metrics_batch(correct[key], layer_random_indep[key])
    result_cascading = run_all_distance_metrics_batch(correct[key], layer_random_cascading[key])
    result_labels = run_all_distance_metrics_batch(correct[key], labels_random[key])
    
    results["correct_layer_random_indep"][key] = result_indep
    results["correct_layer_random_cascading"][key] = result_cascading
    results["correct_labels_random"][key] = result_labels

# Now results is a dictionary where each value is a dictionary of the results of your function
# for each pair of arrays in the corresponding dictionaries.
